# ボトルネック割当問題

$$
\mathrm{minimize}_{\pi: V\to V} \max_{i\in V} c_{i, \pi(i)}
$$

MIPソルバで解く場合は，$z = \max_{i\in V} c_{i, \pi(i)}$ を満たす連続変数 $z$ を導入し，$z$ を最小化する．

解 $\pi$ に対応する変数 $x_{ij} \ (\sum_i x_{ij} = \sum_j x_{ij} = 1)$ について，$c_{i, \pi(i)} = \sum_j c_{i,j} x_{ij}$ に注意すると，
$z$ に課す制約は
$$
z \ge \sum_j c_{i,j} x_{ij} \ \forall i
$$
となる．

In [1]:
import numpy as np
from pyscipopt import Model, quicksum

n = 100 # 1000 だと全然終わらない (200でも数分以上かかる)
cost = np.random.randint(100, 1000, size=(n, n))

V = list(range(n))
model = Model("bap")
x = {}
for i in V:
    for j in V:
        x[i, j] = model.addVar(vtype="B", name=f"x[{i},{j}]")
z = model.addVar(vtype="C", name="z")

for i in V:
    model.addCons(quicksum(cost[i, j] * x[i, j] for j in V) <= z)

for j in V:
    model.addCons(quicksum(x[i, j] for i in V) == 1)
for i in V:
    model.addCons(quicksum(x[i, j] for j in V) == 1)

model.setObjective(z, sense='minimize')

model.setParam("limits/time", 300.0)  # 計算時間の上限を設定

model.optimize() 
print("obj=", model.getObjVal())

presolving:
obj= 143.0
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 1 chg bounds, 0 chg sides, 0 chg coeffs, 200 upgd conss, 0 impls, 200 clqs
   (0.1s) sparsify aborted: 207/30100 (0.7%) nonzeros canceled - in total 207 canceled nonzeros, 9600 changed coefficients, 0 added nonzeros
(round 2, exhaustive) 0 del vars, 0 del conss, 0 add conss, 1 chg bounds, 0 chg sides, 9600 chg coeffs, 200 upgd conss, 0 impls, 200 clqs
   (0.3s) probing: 1000/10000 (10.0%) - 0 fixings, 0 aggregations, 972 implications, 0 bound changes
   (0.3s) probing: 1001/10000 (10.0%) - 0 fixings, 0 aggregations, 973 implications, 0 bound changes
   (0.3s) probing aborted: 1000/1000 successive useless probings
   (0.3s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.3s) no symmetry present (symcode time: 0.00)
presolving (3 rounds: 3 fast, 3 medium, 3 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 5 tightened bounds, 0 add

閾値を設定して通常の割当問題の実行可能解を繰り返し求めることでも解ける
(二分探索しても良いがたいてい下から探すので十分早いらしい)

In [2]:
from scipy.optimize import linear_sum_assignment

In [3]:
n = 1000
cost = np.random.randint(100, 1000, size=(n, n))

In [4]:
%%time
LB = max(cost.min(axis=1).max(), cost.min(axis=0).max() )
row_ind, col_ind = linear_sum_assignment(cost)
UB = cost[row_ind, col_ind].max()
print(f'{LB=}, {UB=}')
for t in range(LB, UB):
    print("Trial with t =", t)
    c = cost.flatten()
    c = np.where(c > t, np.inf, c)
    c.shape=(n,n)
    try:
        row_ind, col_ind = linear_sum_assignment(c)
        break
    except:
        continue
print("Optimum:", cost[row_ind, col_ind].max())

LB=np.int64(106), UB=np.int64(107)
Trial with t = 106
Optimum: 106
CPU times: user 40.5 ms, sys: 3.57 ms, total: 44 ms
Wall time: 43.6 ms
